Wild Blueberry Yield Prediction
Wild blueberry yield prediction using machine learning 

Kaggle Link: https://www.kaggle.com/datasets/saurabhshahane/wild-blueberry-yield-prediction

# 1. Imports

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import cut_the_tails as ct
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
#from sklearn.model_selection import LeaveOneOut,KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data_sets\\Blueberry_Yield.csv')
target = 'yield'

In [3]:
features = ['clonesize',
 'honeybee',
 'bumbles',
 'andrena',
 'osmia',
 'MaxOfUpperTRange',
 'MinOfUpperTRange',
 'AverageOfUpperTRange',
 'MaxOfLowerTRange',
 'MinOfLowerTRange',
 'AverageOfLowerTRange',
 'RainingDays',
 'AverageRainingDays',
 'fruitset',
 'fruitmass',
 'seeds']

In [4]:
fig = px.histogram(df, x=target, nbins=40)
fig.show()

# Cauda Ótima

In [5]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

model = RandomForestRegressor(max_depth=5, random_state=0) 
classifier = RandomForestClassifier(max_depth=5, random_state=0)

x,fval = ct.get_cuts_direct_optimization(df, target, features, classifier, model, 'brute')

print("Final Result: ")
print(x, fval)

[0. 0.] 0.026441964193840956
[0.   0.05] 0.02582032485672226
[0.  0.1] 0.026119376685081883
[0.   0.15] 0.025415738178877084
[0.  0.2] 0.024585625372968146
[0.   0.25] 0.025392076324746608
[0.  0.3] 0.024985229442648204
[0.   0.35] 0.024819299900485307
[0.  0.4] 0.024223006713307096
[0.   0.45] 0.024147038805370117
[0.  0.5] 0.024984441785418792
[0.   0.55] 0.025219819933026043
[0.  0.6] 0.025124566782456472
[0.   0.65] 0.025330167825592158
[0.  0.7] 0.026854697151693443
[0.   0.75] 0.025708297613232355
[0.  0.8] 0.025691374976088897
[0.   0.85] 0.02627046202330284
[0.  0.9] 0.0258570119821097
[0.   0.95] 0.026289026077138268
[0.   0.05] 0.02582032485672226
[0.05 0.05] 0.025838810094153047
[0.05 0.1 ] 0.02703194448913987
[0.05 0.15] 0.025891060003000864
[0.05 0.2 ] 0.02584347922064324
[0.05 0.25] 0.024717378305364968
[0.05 0.3 ] 0.025561463658894168
[0.05 0.35] 0.02458061384426541
[0.05 0.4 ] 0.025378614286851026
[0.05 0.45] 0.02456567670592412
[0.05 0.5 ] 0.02551720959599707
[0.05 0.5

In [6]:
cdf = ct.split_by_quantile_class(df,target,[0.4,0.85])
cdf.drop('Row#', axis=1, inplace=True)

In [7]:
X = cdf[features].to_numpy()
y_tail = cdf['tail_class'].to_numpy()
y = cdf[target].to_numpy()

###Using split###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train_aux, X_test_aux, y_train_tail, y_test_tail = train_test_split(X, y_tail, test_size=0.2, random_state=0)



In [8]:
#baseline = RandomForestRegressor(max_depth=5, random_state=0)
baseline = XGBRegressor(n_estimators=100, max_leaves=0, random_state = 0)
#baseline = DecisionTreeRegressor(random_state=0, max_depth=5) 

In [9]:
#tail_classifier = ct.fit_tail_classifier(X,y_tail,RandomForestClassifier(max_depth=5, random_state=0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 


###Using split###
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,RandomForestClassifier(max_depth=5, random_state=0))
tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 

In [10]:
#models = ct.fit_tail_models(X,y,y_tail,baseline)

###Using split###
models = ct.fit_tail_models(X_train,y_train,y_train_tail,baseline)

In [11]:
#baseline.fit(X,y)
#y_base = baseline.predict(X)

###Using split###
baseline.fit(X_train,y_train)
y_base = baseline.predict(X_test)

In [12]:
#y_tail = ct.batch_tail_predict(X,tail_classifier,models)

###Using split###
y_tail = ct.batch_tail_predict(X_test,tail_classifier,models)

In [13]:
#print(mean_absolute_error(y_base,y))
#print(mean_absolute_error(y_tail,y))
#print(mean_absolute_error(y,y))

###Using split###
print(mean_absolute_error(y_base,y_test))
print(mean_absolute_error(y_tail,y_test))
print(mean_absolute_error(y_test,y_test))

124.54776199338939
115.15781155779244
0.0


In [14]:
#print(mean_absolute_percentage_error(y_base,y))
#print(mean_absolute_percentage_error(y_tail,y))
#print(mean_absolute_percentage_error(y,y))

###Using split###
print(mean_absolute_percentage_error(y_base,y_test))
print(mean_absolute_percentage_error(y_tail,y_test))
print(mean_absolute_percentage_error(y_test,y_test))

0.02329216119407331
0.021082707782797624
0.0
